In [10]:
!pip install opencv-python
!pip install pytest-shutil

### Import libraries

In [11]:
import numpy as np
import pandas as pd
import cv2
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard, ModelCheckpoint
import random
from shutil import copyfile

### Number of images in our Dataset

In [12]:
print("Number of images with mask used: ", len(os.listdir('../dataset/with_mask')))
print("Number of images without mask used: ", len(os.listdir('../dataset/without_mask')))

Number of images with mask used:  692
Number of images without mask used:  686


### Spliting the data into training and test set

In [15]:
def split_data(source, train, test, test_size):
    data = os.listdir(source)
    train_size = int(len(data)*(1 - test_size))
    shuffled_data = random.sample(data, len(data))
    training_data = shuffled_data[:train_size]
    testing_data = shuffled_data[train_size:]
    
    # make new folders train and test
    for img in training_data:
        temp_image = source+img
        train_image = train+img
        copyfile(temp_image, train_image)
    
    for img in testing_data:
        temp_image = source+img
        test_image = test+img
        copyfile(temp_image, test_image)
    

In [17]:
MASK_SOURCE = '../dataset/with_mask/'
TRAIN_MASK_SOURCE = '../dataset/train/yes/'
TEST_MASK_SOURCE = '../dataset/test/yes/'

NO_MASK_SOURCE = '../dataset/without_mask/'
TEST_NO_MASK_SOURCE = '../dataset/test/no/'
TRAIN_NO_MASK_SOURCE = '../dataset/train/no/'

split_data(MASK_SOURCE, TRAIN_MASK_SOURCE, TEST_MASK_SOURCE, 0.2)
split_data(NO_MASK_SOURCE, TRAIN_NO_MASK_SOURCE, TEST_NO_MASK_SOURCE, 0.2)

In [18]:
print('Number of training with mask: ', len(os.listdir(TRAIN_MASK_SOURCE)))
print('Number of training without mask: ', len(os.listdir(TRAIN_NO_MASK_SOURCE)))
print('Number of testing with mask: ', len(os.listdir(TEST_MASK_SOURCE)))
print('Number of testing without mask: ', len(os.listdir(TEST_NO_MASK_SOURCE)))

Number of training with mask:  553
Number of training without mask:  548
Number of testing with mask:  139
Number of testing without mask:  138


### Preprocessing data

In [19]:
TRAIN_DIR = '../dataset/train/'
TEST_DIR = '../dataset/test/'
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory(TRAIN_DIR,
                                                 target_size = (150,150),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory(TEST_DIR,
                                            target_size = (150,150),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 1101 images belonging to 2 classes.
Found 277 images belonging to 2 classes.


### Building CNN

In [21]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu', input_shape=[150, 150, 3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dropout(0.5))

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


### Training the CNN

In [22]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [24]:
# Used to get best weights with monitored with validation loss
checkpoint = ModelCheckpoint('model-{epoch:03d}.model', monitor='val_loss', verbose=0, save_best_only=True, mode='auto')

In [25]:
model = cnn.fit(x = training_set, validation_data = test_set, epochs = 30, callbacks=[checkpoint])

Epoch 1/30
35/35 [==============================] - 93s 3s/step - loss: 0.9161 - acc: 0.5649 - val_loss: 0.3634 - val_acc: 0.9025
Epoch 2/30
35/35 [==============================] - 52s 1s/step - loss: 0.3626 - acc: 0.8538 - val_loss: 0.1423 - val_acc: 0.9603
Epoch 3/30
35/35 [==============================] - 53s 2s/step - loss: 0.3132 - acc: 0.8765 - val_loss: 0.1264 - val_acc: 0.9531
Epoch 4/30
35/35 [==============================] - 54s 2s/step - loss: 0.2998 - acc: 0.8910 - val_loss: 0.1211 - val_acc: 0.9495
Epoch 5/30
35/35 [==============================] - 54s 2s/step - loss: 0.2742 - acc: 0.9019 - val_loss: 0.1049 - val_acc: 0.9639
Epoch 6/30
35/35 [==============================] - 58s 2s/step - loss: 0.1896 - acc: 0.9273 - val_loss: 0.0911 - val_acc: 0.9747
Epoch 7/30
35/35 [==============================] - 54s 2s/step - loss: 0.1856 - acc: 0.9273 - val_loss: 0.1425 - val_acc: 0.9422
Epoch 8/30
35/35 [==============================] - 55s 2s/step - loss: 0.1881 - acc: 0.92

In [27]:
'''
# To save the model in your disk

from keras.models import model_from_json

# serialize model to JSON
model_json = cnn.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn.save_weights("model.h5")
print("Saved model to disk")
'''

Saved model to disk


In [ ]:
# Uncomment this to directly load the trained model if you don't wish to train it on your computer
'''
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
'''